### Prepare the datasets from the annotations

In [1]:
from tqdm import tqdm_notebook as tqdm
import json
import numpy as np
import os
import re

In [ ]:
annotations = [] 
with open('annotations_2000.json1','r') as f: 
    for line in f.readlines():
        annotations.append(json.loads(line))

In [ ]:
tot = 0
nothing = 0 
valid_annotations = []
for samp in annotations:
    if len(samp['labels']) == 0:
        continue
    if samp['labels'][0][2] == 'Nothing':
        nothing += 1
        continue
    valid_annotations.append(samp)

In [ ]:
def get_initVecs(i):
    text = valid_annotations[i]['text']
    text = text.strip(".")
    y_small = ['0' for i in range(len(text))]
    
    for ran in sorted(valid_annotations[i]['labels']):
        if ran[2] == 'Inclusion':
            arr = ['1']*(ran[1] - ran[0]+1)
        else:
            arr = ['2']*(ran[1]-ran[0]+1)
        y_small[ran[0]:ran[1]+1] = arr

    for i in range(len(text)):
        if text[i] == '.': y_small[i] = '.'
        if text[i] == " ":y_small[i] = " "
    y_string = "".join(y_small)
    y_string = y_string.split('.')
    y = []
    sentences = text.split('.')
    for i in range(len(sentences)):
        if len(y_string[i]) == 0:
            continue
        temp = []
        for wrd in y_string[i].split(" "):
            if len(wrd) == 0:continue
            temp.append(int(wrd[0]))
        y.append(temp)
    for i in range(len(sentences)):
        sentences[i] = sentences[i].strip(' ')
    sentences = [sent for sent in sentences if len(sent)>0]
    return sentences, np.array(y)

In [ ]:
data = []
sentid = -1
for i in range(len(valid_annotations)):
    sentences, labels = get_initVecs(i)
    for j in range(len(sentences)):
        if sum(labels[j])==0:
            continue
        sentid += 1
        sents = sentences[j].split(" ")
        for k in range(len(labels[j])):
            temp = {}
            temp['id'] = sentid
            temp['words'] = sents[k]
            if labels[j][k] == 0:
                temp['labels'] = 'O'
            elif labels[j][k] == 1:
                temp['labels'] = 'INC'
            else:
                temp['labels'] = 'EXC'
            data.append(temp)
df = pd.DataFrame(data)

In [ ]:
def clean_data(l):
    
    word = str(l['words'])
    word = word.lower()
    word = re.sub('[^a-zA-Z0-9]','',word)
    return word

df['cleaned'] = df.apply(lambda l : clean_data(l),axis=1)
df.rename({'cleaned':'words'},axis=1,inplace=True)
df = df[['id','words','labels']]
df.to_csv('Task1.csv')

### Prepare Data for Second Level Classification

In [ ]:
import pandas as pd
df = pd.read_csv('Task1.csv')
sentences = []
cur = []
typ = ''
ind = 0

for i,row in df.iterrows():
    if row['labels'][0] == 'B' and len(cur):
        cur = [str(c) for c in cur]
        sentences.append({'id':ind,'sentence':' '.join(cur),'type':typ})
        cur = []
    if row['labels'] != 'O':
        if row['labels'][0] == 'B':
            ind = row['id']
        if 'EXC' in row['labels']:
            typ = 'E'
        else:
            typ = 'I'
        cur.append(row['words'])
        
cur = [str(c) for c in cur]
sentences.append({'id':ind,'sentence':' '.join(cur)})

In [ ]:
task2 = pd.DataFrame(sentences)
task2.to_csv('Task2.csv')